In [ ]:
import os
import pandas as pd
from PIL import Image
from io import BytesIO
import base64

def get_image_metadata(folder_path):
    data = []
    valid_extensions = {".jpg", ".png"}
    
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        if not any(file_name.lower().endswith(ext) for ext in valid_extensions):
            continue
        
        try:
            with Image.open(file_path) as img:
                color_model = img.mode
                img_format = img.format
                num_channels = len(img.getbands())
                width, height = img.size
                file_size_mb = round(os.path.getsize(file_path) / (1024 * 1024), 2)
                
                img.thumbnail((100, 100))
                buffer = BytesIO()
                img.save(buffer, format="PNG")
                img_data = base64.b64encode(buffer.getvalue()).decode()
                img_html = f'<img src="data:image/png;base64,{img_data}" width="100" height="100">'
                
                data.append([
                    file_name, color_model, img_format, num_channels, file_size_mb,
                    width, height, file_path, img_html
                ])
        except Exception as e:
            print(f"Помилка обробки {file_name}: {e}")
            
    columns = [
        "Ім'я файлу", "Кольорова модель", "Формат", "Кількість каналів",
        "Розмір файлу (МБ)", "Ширина", "Висота", "Повний шлях", "Зображення"
    ]
    
    df = pd.DataFrame(data, columns=columns)
    return df.style.format({"Зображення": lambda x: x})

get_image_metadata("files")

In [ ]:
import os
import random
from PIL import Image, ImageEnhance, ImageOps

def crop_to_square(image):
    width, height = image.size
    min_side = min(width, height)
    left = (width - min_side) // 2
    top = (height - min_side) // 2
    right = left + min_side
    bottom = top + min_side
    return image.crop((left, top, right, bottom))

def apply_random_filter(image):
    filters = {
        "red": (255, 0, 0),
        "green": (0, 255, 0),
        "blue": (0, 0, 255),
        "yellow": (255, 255, 0),
        "magenta": (255, 0, 255),
        "cyan": (0, 255, 255)
    }
    
    filter_color = random.choice(list(filters.values()))
    overlay = Image.new("RGB", image.size, filter_color)
    return Image.blend(image.convert("RGB"), overlay, alpha=0.3)

def generate_poster(folder_path, output_folder, output_filename, columns, rows, image_size=500, num_variants=3):
    images = [f for f in os.listdir(folder_path) if f.lower().endswith((".jpg", ".png"))]
    
    if not images:
        raise ValueError("У папці немає зображень.")
    
    os.makedirs(output_folder, exist_ok=True)
    
    for i in range(num_variants):
        poster = Image.new("RGB", (columns * image_size, rows * image_size))
        
        for row in range(rows):
            for col in range(columns):
                img_path = os.path.join(folder_path, random.choice(images))
                with Image.open(img_path) as img:
                    img = crop_to_square(img).resize((image_size, image_size))
                    img = apply_random_filter(img)
                    poster.paste(img, (col * image_size, row * image_size))
        
        output_path = os.path.join(output_folder, f"{output_filename}_{i+1}.jpg")
        poster.save(output_path)

def show_posters(results_folder):
    images = [f for f in os.listdir(results_folder) if f.lower().endswith((".jpg", ".png"))]
    
    if not images:
        print("У папці", results_folder, "немає зображень.")
        return
    
    for img_name in images:
        img_path = os.path.join(results_folder, img_name)
        display(Image.open(img_path))
        print("\n")

generate_poster("files", "results", "poster", 4, 3)
show_posters("results")